In [1]:
import sys
import math
import numpy as np
import argparse
import scipy.io as sio
import scipy.linalg as spla

from config import Vxf0, Options

# path imports
sys.path.insert(0, "CLFM_lib")
sys.path.insert(1, "GMR_lib")

def loadSavedMatFile(x):
    matFile = sio.loadmat(x)

    # print(matFile)
    data = matFile['Data']
    demoIdx = matFile['demoIndices']

    return data, demoIdx

def guess_init(data, Vxf0):
    pass

modelNames = ['w.mat', 'Sshape.mat']   # Two example models provided by Khansari
modelNumber = 1  # could be zero or one depending on the experiment the user is running

data, demoIdx = loadSavedMatFile('ExampleModels/' + modelNames[modelNumber])

Vxf0['L'] = modelNumber
Vxf0['d'] = int(data.shape[0]/2)
Vxf0['Mu'] =  np.empty([2, 2, 2])
Vxf0['P'] =  np.empty([2,2,2])
temp = np.transpose(data[0:Vxf0['d'],:])
tempvar = np.var(temp, axis=0)
lengthScale = np.sqrt(tempvar)
tempcov = np.cov(temp, rowvar=False)
lengthScaleMatrix = spla.sqrtm(tempcov)

Vxf0['Priors'] = np.random.rand(Vxf0['L']+1,1);
# print('demoIdx: ', demoIdx)
# print(Vxf0['d'], data.shape[0])
# print(temp.shape)
# print(tempvar)
# print(tempcov)
# print (Vxf0['Priors'])
print(Vxf0['d'])
print((np.random.randn(Vxf0['d'],1)*lengthScale).shape)
for l in range(Vxf0['L']+1):
    tempMat = np.random.randn(Vxf0['d'], Vxf0['d'])
    Vxf0['Mu'][:,l] = np.random.randn(Vxf0['d'],1)*lengthScale
    Vxf0['P'][:,:,l] = lengthScaleMatrix * (tempMat * np.transpose(tempMat)) * lengthScaleMatrix

for k,v in Vxf0.items():
    print(k,v)

2
(2, 2)
L 1
d 2
w 0.0001
Mu [[[-16.72155633 -31.58215363]
  [-48.30344752 -91.23115521]]

 [[-19.71813816 -37.24182465]
  [-32.37705438 -61.15083343]]]
P [[[  7.10486780e+03   3.63919421e+01]
  [ -4.46248161e-01   3.72939987e-01]]

 [[ -4.46248161e-01   3.72939987e-01]
  [  1.31222108e+04   9.77701563e+03]]]
Priors [[ 0.7716872 ]
 [ 0.00335896]]


In [58]:
Vxf0['Priors'] = np.ones((Vxf0['L']+1, 1))
Vxf0['Priors'] = Vxf0['Priors']/sum(Vxf0['Priors'])
Vxf0['Mu'] = np.zeros((Vxf0['d'],Vxf0['L']+1))
for l in range(Vxf0['L']+1):
    Vxf0['P'][:,:,l] = np.eye((Vxf0['d']))

In [66]:
Vxf0['Mu']  =  np.empty([ Vxf0['d'], Vxf0['L']+1 ]) #will be 2x2
Vxf0['P']   =  np.empty([ Vxf0['d'], Vxf0['d'], Vxf0['L']+1 ]) # wil be 2x2x3


array([[ 1.,  0.],
       [ 0.,  1.]])

In [109]:
a = np.zeros((2,2))
print(a)
ar = a.reshape(1, a.size)
print(ar.shape)

[[ 0.  0.]
 [ 0.  0.]]
(1, 4)


In [98]:
a = np.array([[1, 2], [3, 4]])
b = np.array([[5, 6]])
c = np.concatenate( (a, b.T), axis=1)
print(c, b.T)

[[1 2 5]
 [3 4 6]] [[5]
 [6]]


In [110]:
help(np.array)

Help on built-in function array in module numpy.core.multiarray:

array(...)
    array(object, dtype=None, copy=True, order=None, subok=False, ndmin=0)
    
    Create an array.
    
    Parameters
    ----------
    object : array_like
        An array, any object exposing the array interface, an
        object whose __array__ method returns an array, or any
        (nested) sequence.
    dtype : data-type, optional
        The desired data-type for the array.  If not given, then
        the type will be determined as the minimum type required
        to hold the objects in the sequence.  This argument can only
        be used to 'upcast' the array.  For downcasting, use the
        .astype(t) method.
    copy : bool, optional
        If true (default), then the object is copied.  Otherwise, a copy
        will only be made if __array__ returns a copy, if obj is a
        nested sequence, or if a copy is needed to satisfy any of the other
        requirements (`dtype`, `order`, etc.).

In [134]:
a = np.matrix(([2,1,3], [5,4,6]))
# print(a.shape)
b = np.sort(a)
d = np.ones((5,2))
e = np.ones(2)


In [138]:
X = np.empty((3,3))
empty = np.array(())
print(X.reshape(9, empty))

TypeError: only length-1 arrays can be converted to Python scalars

In [139]:
help(np.reshape)

Help on function reshape in module numpy.core.fromnumeric:

reshape(a, newshape, order='C')
    Gives a new shape to an array without changing its data.
    
    Parameters
    ----------
    a : array_like
        Array to be reshaped.
    newshape : int or tuple of ints
        The new shape should be compatible with the original shape. If
        an integer, then the result will be a 1-D array of that length.
        One shape dimension can be -1. In this case, the value is inferred
        from the length of the array and remaining dimensions.
    order : {'C', 'F', 'A'}, optional
        Read the elements of `a` using this index order, and place the elements
        into the reshaped array using this index order.  'C' means to
        read / write the elements using C-like index order, with the last axis
        index changing fastest, back to the first axis index changing slowest.
        'F' means to read / write the elements using Fortran-like index order,
        with the firs